In [1]:
import pandas as pd

In [2]:
fpath = "Resources/purchase_data.csv"
hop_df = pd.read_csv(fpath,encoding = 'latin1')
hop_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [3]:
player_list = hop_df[['SN','Gender','Age']].drop_duplicates()
player_count = len(player_list['SN'])
#print(f'Total number of players: {player_count}')
pc_df = pd.DataFrame([{"Total Players":player_count}])
pc_df

,Total Players
0,576


## Purchasing Analysis (Total)
* Number Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [4]:
item_count = hop_df['Item Name'].nunique()
# print(f'Number of Unique Items:{item_count}')

ave_purch = hop_df['Price'].mean()
# print(f'Average Purchase Price: ${ave_purch:.2f}')

tot_purch_count = hop_df['Purchase ID'].nunique()
# print(f'Total Number of Purchases: {tot_purch_count}')

tot_rev = hop_df['Price'].sum()
# print(f'Total Revenue: ${tot_rev:.2f}')

purch_analysis = {"Num of Unique Items":item_count,
                  "Average Price":ave_purch,
                  "Num of Purchases":tot_purch_count,
                  "Total Revenue": tot_rev}
pa_df = pd.DataFrame(purch_analysis,index = [0])
pa_df

,Num of Unique Items,Average Price,Num of Purchases,Total Revenue
0,179,3.050987,780,2379.77


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [5]:
pcount_m = player_list['Gender'].value_counts()['Male']
pcount_f = player_list['Gender'].value_counts()['Female']
pcount_o = player_list['Gender'].value_counts()['Other / Non-Disclosed']

perc_m = round(pcount_m/player_count,4)
perc_f = round(pcount_f/player_count,4)
perc_o = round(pcount_o/player_count,4)

gender_dem = {"Count":[pcount_f,pcount_m,pcount_o],
              "Percentage of Players":[perc_f,perc_m,perc_o]}
gd_df = pd.DataFrame(gender_dem, index = ["Female","Male","Other/Non-Disclosed"])
gd_df["Percentage of Players"] = gd_df["Percentage of Players"].map('{:.2%}'.format)
gd_df

,Count,Percentage of Players
Female,81,14.06%
Male,484,84.03%
Other/Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [6]:
grouped_hop_df = hop_df.groupby(['Gender'])

purch_by_gend = grouped_hop_df['Gender'].count()
avepurch = grouped_hop_df['Price'].mean()
purchtotals = grouped_hop_df['Price'].sum()
avepurch_per = (purchtotals/gender_dem['Count'])    

pa_gen = {
    "Purchase Count":purch_by_gend,
    "Average Purchase": avepurch,
    "Total Purchase Value":purchtotals,
    "Avg Total Purchase per Person":avepurch_per
    }

summ_pagen_df = pd.DataFrame(pa_gen,index = ["Female","Male","Other / Non-Disclosed"])
summ_pagen_df = summ_pagen_df.style.format({"Average Purchase":"${:,.2f}","Total Purchase Value":"${:,.2f}", "Avg Total Purchase per Person":"${:,.2f}"})
summ_pagen_df


,Purchase Count,Average Purchase,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table

In [7]:
bins = [5,9,14,19,24,29,34,39,50]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

player_list['Age Group'] = (pd.cut(player_list['Age'],bins,labels=group_names))
player_list_groupby = player_list.groupby('Age Group')
player_age_count = player_list_groupby.count()['Age']
player_age_perc = player_age_count/player_count

player_age_summary = {
    'Total Count':player_age_count,
    'Percentage of Players':player_age_perc
}
player_age_sum_df = pd.DataFrame(player_age_summary)
player_age_sum_df = player_age_sum_df.style.format({"Percentage of Players":"{:.2%}"})
player_age_sum_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (age)

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age

In [8]:
hop_df['Age Group'] = (pd.cut(hop_df['Age'],bins,labels=group_names))
purch_by_age = hop_df.groupby('Age Group')
pba_count = purch_by_age.count()['Purchase ID']
pba_ave = purch_by_age.mean()['Price']
pba_tot_rev = purch_by_age.sum()['Price']
pba_ave_per = pba_tot_rev/player_age_count
age_purch_summ = {
    'Purchase Count':pba_count,
    'Average Purchase Price':pba_ave,
    'Total Purchase Value':pba_tot_rev,
    'Avg Total Purchase per Person':pba_ave_per
}
age_purch_df = pd.DataFrame(age_purch_summ)
age_purch_df = age_purch_df.style.format({"Average Purchase Price":"${:,.2f}",
                                          "Total Purchase Value":"${:,.2f}",
                                          'Avg Total Purchase per Person':"${:,.2f}"})
age_purch_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [9]:
hop_grp_sn = hop_df.groupby('SN')
sn_purch_count = pd.DataFrame(hop_grp_sn.count()['Purchase ID'])
sn_tot = pd.DataFrame(hop_grp_sn.sum()['Price'])

sn_df = pd.merge(sn_tot,sn_purch_count,on="SN",how='outer')
sn_df = sn_df.sort_values(['Price'], ascending=False)
sn_df = sn_df.rename(columns={'Price':"Total Purchase Value","Purchase ID":"Purchase Count"})
sn_topfive_df = sn_df[0:5].copy()

sn_topfive_df["Average Purchase Price"] = (sn_topfive_df['Total Purchase Value']/sn_topfive_df['Purchase Count'])
sn_topfive_df = sn_topfive_df[['Purchase Count','Average Purchase Price','Total Purchase Value']]
sn_topfive_df = sn_topfive_df.style.format({"Average Purchase Price":"${:,.2f}",
                                          "Total Purchase Value":"${:,.2f}"})
sn_topfive_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value
  
* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
# Retrieve Item ID, Item Name, and Item Price columns
item_table_df = hop_df[['Item ID', 'Item Name', 'Price']].copy()#.set_index('Item ID')
# Group by Item ID and Item Name
item_table_grp = item_table_df.groupby(['Item ID','Item Name'])

#Calculations
purch_count = pd.DataFrame(item_table_df['Item ID'].value_counts())
purch_count.rename(columns = {'Item ID':'Count'},inplace=True)

item_prices = item_table_grp.mean('Price') #DF
tot_value = item_table_grp.sum('Price') #DF

#Summary Frame
#item_summ = pd.merge(purch_count,item_prices,on='Item ID',how='inner')
    #'Item ID':purch_count.loc[0:5,'Item ID'],
    #'Item Name':item_table_df.loc[,'Item Name']
    #'Purchase Count':purch_count.loc[0:5,'Count'],
    #'Average Price':item_prices
#})

#top_five_summ
item_prices


,,Price
Item ID,Item Name,
0,Splinter,1.2800
1,Crucifer,2.9425
2,Verdict,2.4800
3,Phantomlight,2.4900
4,Bloodlord's Fetish,1.7000
...,...,...
178,"Oathbreaker, Last Hope of the Breaking Storm",4.2300
179,"Wolf, Promise of the Moonwalker",4.4800
181,Reaper's Toll,1.6600


### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value